# Generate All Paper Figures

This notebook generates all high-quality figures for the paper.

Figures generated:
1. Basic Π(x), B(x), E(x) curves
2. Error growth in log-log scale
3. Judge comparison
4. Spectrum analysis
5. Zero distribution
6. Complexity distribution

All figures are saved to `../output/figures/` for LaTeX inclusion.


In [1]:
import sys
sys.path.append('..')
import numpy as np
import matplotlib.pyplot as plt

from core.action_space import generate_world
from core.judgement_system import BiasedJudge, NoisyJudge, ConservativeJudge, batch_evaluate
from core.ethical_primes import select_ethical_primes, compute_Pi_and_error, analyze_error_growth, compare_error_distributions
from analysis.zeta_function import build_m_sequence, compute_spectrum, find_approximate_zeros, analyze_spectrum_peaks
from visualization.plots import *

setup_paper_style()
np.random.seed(42)

print("Generating all paper figures...")

# Generate data
actions = generate_world(num_actions=2000, complexity_dist='zipf', random_seed=42)
judges = {
    'Biased': BiasedJudge(bias_strength=0.2),
    'Noisy': NoisyJudge(noise_scale=0.3),
    'Conservative': ConservativeJudge(threshold=0.5)
}
results = batch_evaluate(actions, judges, tau=0.3)

# Figure 1: Basic Pi, B, E curves (Biased Judge)
primes_biased = select_ethical_primes(results['Biased'])
Pi_x, B_x, E_x, x_vals = compute_Pi_and_error(primes_biased, X_max=100)
plot_Pi_B_E(x_vals, Pi_x, B_x, E_x, save_path='../output/figures/paper_fig1_pi_b_e.pdf', show=False)

# Figure 2: Error growth analysis
analysis = analyze_error_growth(E_x, x_vals)
plot_error_growth(x_vals, E_x, analysis, save_path='../output/figures/paper_fig2_error_growth.pdf', show=False)

# Figure 3: Multi-judge comparison
comparison = compare_error_distributions(results, X_max=100)
plot_multi_judge_errors(comparison, save_path='../output/figures/paper_fig3_judge_comparison.pdf', show=False)

# Figure 4: Spectrum
m = build_m_sequence(primes_biased, X_max=200)
freqs, amps = compute_spectrum(m)
peaks = analyze_spectrum_peaks(freqs, amps)
plot_spectrum(freqs, amps, peaks, save_path='../output/figures/paper_fig4_spectrum.pdf', show=False)

# Figure 5: Zero distribution
zeros = find_approximate_zeros(m, grid_size=40)
plot_zero_distribution(zeros, save_path='../output/figures/paper_fig5_zeros.pdf', show=False)

# Figure 6: Complexity distribution
plot_complexity_distribution(actions, save_path='../output/figures/paper_fig6_complexity_dist.pdf', show=False)

print("All figures generated successfully!")
print("Output location: ../output/figures/")


Generating all paper figures...
All figures generated successfully!
Output location: ../output/figures/
